<a href="https://colab.research.google.com/github/rudevico/Gachon-AISTUDY/blob/main/12_Ensemble_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 예제11까지 다뤄온 Data
## 0. 1. Structured data; 정형 데이터
CSV, Database, Excel 등으로 다룰 수 있는 data를 의미한다.  
대부분의 데이터는 Structured data에 속한다.
또한 예제11까지 다룬 ML algorithm들은 Structured data에 적용하기 적합하다.  
이때 Structured data를 다루는 대부분의 경우에서 가장 뛰어난 성과를 내는 algorithm이 있는데, ***Ensemble learning**이다.  
> *In statistics and machine learning, __ensemble methods__ use mulitple learning algorithms to obatin better perdictive performeance than could be obtained from any of the constituent learning algorithms alone. - [Wikipedia](https://en.wikipedia.org/wiki/Ensemble_learning)

## 0. 2. Unstructured data; 비정형 데이터
텍스트 데이터, 사진, 디지털 음악 등이 여기에 속한다.
> Unstructured data의 경우 규칙성을 찾기가 어려워서 기존의 machine learning 방법으로는 model을 만들기가 까다롭다.  
때문에 나중에 배울 **neural network algorithm**을사용한다.

## 0. 3. Ensemble methods
본 예제 **12_Ensemble-learning.ipynb**에서는 다음과 같은 ensemble methods에 대해서 다룬다.  
* #1. Random Forest
* #2. Extra Trees
* #3. Gradient Boosting
* #4. Histogram-based Gradient Boosting

# 1. Random Forest; 랜덤 포레스트

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [ ]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [ ]:
wine.describe()

,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


In [ ]:
X = wine.drop('class', axis=1).to_numpy()
y = wine['class'].to_numpy()
print(X[:3])
print(y[:3])

[[9.4  1.9  3.51]
 [9.8  2.6  3.2 ]
 [9.8  2.3  3.26]]
[0. 0. 0.]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# case1. test score만 리턴
# scores = cross_validate(rf, X_train, y_train,
#                         n_jobs=-1)
# print(np.mean(scores['test_score']))
# print(scores)

# case2. train score도 리턴
# 여기서의 test는 validation set임을 유의
scores = cross_validate(rf, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
print(scores)

0.9973541965122431 0.8905151032797809
{'fit_time': array([2.08594203, 2.17877221, 1.83536553, 1.90284824, 1.263165  ]), 'score_time': array([0.13418102, 0.15252733, 0.15143609, 0.11618233, 0.12137032]), 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}


위 cell을 보면 overfitting된 것을 확인할 수 있다.  
본 예제에서는 RF algorithm의 성능을 높이는 것이 주 목적이 아니라, 학습 자체가 목적이기 때문에 추가적인 handling은 하지 않는다.

Random Forest는 결국 Decision Tree의 ensemble이므로 기본적으로 `DecisionTreeClassifier()`에서의 주요 *parameters를 그대로 사용한다.
> `criterion`, `max_depth`, `max_features`, `min_samples_split`, `min_impurity_decrease`, `min_samples_leaf` 등.  
또한 Decision Tree의 큰 장점 중 하나인 `feature_importances_` attribute도 계산한다.  

> 이때 RF에서의 `feature_importances_` attribute의 값은 RF를 이루는 각 DT의 `feature_importances_` attribute 값을 취합한 것이다.

In [ ]:
rf.fit(X_train, y_train)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


앞선 **10_Decision-Tree.ipynb** 예제에서의 `dt.feature_importances_`는 각각 다음과 같았다.  
```[0.12345626 0.86862934 0.0079144  ]```  
둘을 비교해보면 RF에서의 `feature_importances_` attribute의 값이 보다 고르게 퍼진 것을 확인할 수 있다.  
> 기본적으로 DT에서는 **_entire features 중에서_** child node의 impurity decrease 정도를 가장 크게 만드는 feature를 선택한다.  
또한 대부분의 경우에서는 entire features가 동등하게 중요하지는 않다(ex. wine의 경우에는 sugar만 중요하고, 나머지 features는 사실상 의미가 없었다).  
이 때문에 대부분의 node에서는 most important feature(sugar)에 대해서만 나눠지게 되는데 이는 overfitting을 유발할 수 있다.  

> 기본적으로 RF에서는 먼저 각 node마다 entire features 중에 일부를 일단 고른다(default는 **squared roof** of number of entire features 즉, 전체 특성 수가 9개라면 그 제곱근인 3개를 각 node마다 고름, 또한 소수점의 경우 내림).  
  >> 그리고 **_선택된 일부의 features 중에서_** child node의 impurity decrease 정도를 가장 크게 만드는 feature를 선택한다.  
  따라서 most importance feature라고 하더라도, '일부'에 선택되지 못한다면 사용되지 못한다.  
  따라서 그 하위의 importance를 가진 features에게 기회가 주어진다.  
  따라서 하나의 feature에 과도하게 집중하지 않고 좀 더 많은 features를 사용하여 훈련하게 된다.

* * *
`RandomForestClassifier()` 클래스는 **자체적으로 model을 평가하는 점수**를 얻을 수도 있다.  
RF에서는 **중복을 허용하는** bootstrap sample을 사용한다.  
따라서 확률적으로 어떤 sample은 중복으로 선택될 것이고, ***어떤 sample은 아예 선택되지 않을 것**이다.  
> *이를 __$OOB^{out of bag}$ sample__이라고 한다.  

즉 RF를 이루는 각 DT는 bootstrap sample로 training되고, training된 DT를 OOB로 평가할 수 있다(validation set과 유사한 개념).

In [ ]:
# default, oob_score=False. n_jobs=1
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)
print(rf.oob_score_)

0.8934000384837406


> 현재까지 **X_test, y_test는 사용한 적이 없음**을 명심하자.

> `np.mean(scores['test_score']`는 X_test가 아니라, RF(또는 DT)에 대해서 수행하는 cross_validation(k-fold)에서의 test 즉, **validation set에 대한 score**이다.  
>> k-fold cross validation에서는 sub_train이 train이고 validation이 test 개념이기 때문

> `rf.oob_score_`는 **OOB sample에 대한 score**이다.

# 2. Extra Trees; 엑스트라 트리
Extra Trees는 다음과 같은 특징을 갖는다.  
* __DT 및 RF와 유사하게__ DT가 제공하는 대부분의 parameters 지원
* __RF와 유사하게__ entire features 중에서 일부 특성을 random하게 선택  
  - RF는 일부 특성 몇 개를 random하게 선택하고, 그 중에서 최적의 분할을 찾는다.
  - ET는 entire features 중에 1개를 random하게 선택해서 분할한다.
* __RF와는 다르게__ bootstrap sample을 사용하지 않는다  
  즉, 각 Decision Tree를 만들 때 train set의 전체를 사용한다.  
* __RF와는 다르게__ ***가장 좋은 분할**을 찾는 것이 아니라 무작위로 분할한다.  
  > *child node의 impurity decrease 정도가 가장 크게끔 분할되는 것을 의미한다.  
    `ExtraTreesClassifier()`는 각 DT에 대해서 `splitter` parameter를  'random'으로 설정한다.  
    다음 cell을 보면 DT의 default splitter는 'best'임을 확인할 수 있다.  

  - 무작위로 분할한다면 각 node가 최적으로 분할되지 않기 때문에 **model performance는 낮아**질 수 있다.
  - 다만 특정 feature가 과도하게 선택되지 않기 때문에 **overfitting을 방지**할 수 있다.
  - 또한 경우의 수를 나열하고 그 중에서 최적의 node를 찾는 방식이 아니라, random하게 node를 분할하기 때문에 **연산 과정이 크게 감소**한다.


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
print(dt.splitter)

best


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
print('RF에서의 score는 다음과 같았다.\n0.9973541965122431 0.8905151032797809')

0.9974503966084433 0.8887848893166506
RF에서의 score는 다음과 같았다.
0.9973541965122431 0.8905151032797809


* RF에서는 entire features 중에서 일부를 랜덤하게 선택하고, 그 중에서 최적의 분할을 찾는 방식으로 DT에서 most importance feature에 과도하게 치중되는 문제를 해소했다.  
* ET에서는 random하게 node를 분할하는 방식으로 위 문제도 해소하고, 연산 속도도 크게 감소시켰다.

In [ ]:
et.fit(X_train, y_train)
print('alcohol\t\tsugar\tpH')
print(et.feature_importances_)
print('DT에서의 feature_importances_는 다음과 같았다.\n[0.12345626 0.86862934 0.0079144 ]')
print('RF에서의 feature_importances_는 다음과 같았다.\n[0.23167441 0.50039841 0.26792718]')

alcohol		sugar	pH
[0.20183568 0.52242907 0.27573525]
DT에서의 feature_importances_는 다음과 같았다.
[0.12345626 0.86862934 0.0079144 ]
RF에서의 feature_importances_는 다음과 같았다.
[0.23167441 0.50039841 0.26792718]


# 3. Gradient Boosting; 그레이디언트 부스팅
scikit-learn에서 제공하는 Gradient Boosting은 기본적으로 `max_depth=3`, `n_estimators=100`, `learning_rate=0.1`이다.  
즉 X와 y가 존재할 때 Model 1은 X에 대한 y를 prediction하고, 이를 실제 y와 비교한다. 이때 두 값의 차이를 **residual; 잔차**라고 한다.  
Model 1에서의 Residual 1은 Model 2의 y가 되고, Model 2는 y를 prediction하고, 이를 실제 y와 비교한다.  
(이하 반복 ...)  
> 따라서 Model n+1을 수행하기 위해서는 Model n이 존재해야 하는 순차적인 학습이다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, X_train, y_train,
                        return_train_score=True, n_jobs=1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


위 cell과 같이 max_depth가 얕은 DT를 여러 개 사용하기 때문에 overfitting이 거의 발생하지 않는다.  

~~Increase: To make something larger in amount, number, or degree.~~  
~~Enhance: To improve the quality, value, or attractiveness of something.~~  
**How to enhance the performance of a gradient boosting model**  
  * Increase the value of the `learning_rate` parameter.
  * Increase the number of trees(the `n_estimators` parameter).

또한 gradient boosting은 순차적 학습을 하기 때문에 `n_jobs` parameter가 존재하지 않음을 확인할 수 있다.

In [ ]:
# error 확인용
print(gb.n_jobs)

AttributeError: 'GradientBoostingClassifier' object has no attribute 'n_jobs'

In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)
scores = cross_validate(gb, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


위 cell을 보면 `n_estimators`와 `learning_rate` parameter의 조정으로 model performance가 향상된 것을 확인할 수 있다.

In [ ]:
gb.fit(X_train, y_train)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


정리하자면, gradient boosting은 다음과 같은 특징을 갖는다.  
* 병렬처리가 어렵다. `n_jobs` parameter가 존재하지 않는다. training이 오래 걸린다.
* `n_estimator`, `learning_rate` 등의 parameter의 수가 다른 ensemble algorithm에 비해 많기 때문에 hyperparameter tuning 또한 더 어렵다.

# 4. Histogram-based Gradient Boosting; 히스토그램 기반 그레이디언트 부스팅
Histogram-based Gradient Boosting은 다음과 같은 특징을 갖는다.  
* 먼저, 입력된 특성(일반적으로 Continuous)을 256개의 discrete한 구간으로 나눈다.  
ex. 입력의 최소 value가 1, 최대 value가 1,000이라면 구간 1 = 1 ~ 4, 구간 2 = 5 ~ 8, ... 구간 256 = 997 ~ 1000
* 각 구간에 속하는 데이터의 수를 구한다(Histogram 형식).
* node를 분할할 때 256개의 구간들에 대해서 각 구간을 기준으로 분할했을 때의 loss function 감소량을 계산하고, 감소량을 가장 크게 만드는 구간을 기준으로 분할한다.
  - 구간 1을 기준으로 나눴을 때의 loss function 감소량 = 2,  
    구간 15를 기준으로 나눴을 때의 loss function 감소량 = 10이고 가장 큰 값이라면, 구간 15를 기준으로 분할한다.
  - 즉 데이터 분포가 얼마나 넓은지에 상관없이 256번만 계산하면 되기 때문에 매우 빠르게 연산할 수 있다.

> 엄밀히 말하자면, 256개 구간 중 한 개의 구간은 missing value가 존재했을 때 넣어두는 구간이다. 따라서 missing value에 대한 전처리를 따로 하지 않아도 되는 편리함이 있다.  
일단은 이 정도로만 알아두면 되겠다.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, X_train, y_train,
                        return_train_score=True) # gradient boosting의 일종이기 때문에 n_jobs 없다.
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


위 cell을 보면 미세하지만 앞선 GradientBoosting보다 train_score는 감소하고, test_score는 증가한 것을 확인할 수 있다.

* * *

In [ ]:
hgb.fit(X_train, y_train)
print(hgb.feature_importances_)

AttributeError: 'HistGradientBoostingClassifier' object has no attribute 'feature_importances_'

(위 cell 참고)또한 scikit-learn의 histogram-based gradient boosting(classifier, regression)에는 특성 중요도를 계산한 값이 보관되는 `feature_importances_` attribute가 존재하지 않는다.  
* **continuous value**를 **discrete value**로 변환했기 때문에 기존의 특성 중요도 계산 방식으로는 정확하게 특성 중요도를 계산해낼 수 없다.
* 이에 대한 대안으로 `permutation_importance()`를 사용하여 특성 중요도를 계산할 수 있다.  

> 위 함수는 다음과 같이 작동한다.  
  * histogram-based gradient boosting을 사용해서 train set으로 model을 training하고, test set으로 model performance를 측정한다.
  * test set에서 각 data instances들의 $ x_1 $값을 shuffling한다.  
    이때 다른 features의 값은 변경하지 않는다.
  * __$ x_1 $ 값만 shuffling된 permutated_1 test set__으로 model performance를 측정한다(일반적으로 감소할 것이다).
  * performance 감소 정도가 크다면 특성 중요도 값을 높게 평가하고, 아니라면 낮게 평가한다.
  * 위 과정을 __$ x_2 $ 값만 shuffling된 permutated_2 test set__, __$ x_3 $ 값만 shuffling된 permutated_3 test set__, ...에 대해서 반복한다.
  * 그러면 $ x_j $ 값이 shuffling 되었을 때의 performance 감소량을 비교하여 특성 중요도 값을 산정할 수 있다.

In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(X_train, y_train)
# defalut, n_repeats=5. shuffle 횟수
result = permutation_importance(hgb, X_train, y_train,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
# permutation_importance 객체에는 중요도 array, mean, std가 있음
print(result)

{'importances_mean': array([0.08876275, 0.23438522, 0.08027708]), 'importances_std': array([0.00382333, 0.00401363, 0.00477012]), 'importances': array([[0.08793535, 0.08350972, 0.08908986, 0.08312488, 0.09274581,
        0.08755051, 0.08601116, 0.09601693, 0.09082163, 0.09082163],
       [0.22782374, 0.23590533, 0.23936887, 0.23436598, 0.23725226,
        0.23436598, 0.23359631, 0.23398114, 0.23994612, 0.22724649],
       [0.08581874, 0.08601116, 0.08062344, 0.07504329, 0.08427939,
        0.07792957, 0.07234943, 0.07465846, 0.08139311, 0.08466423]])}


다음 cell을 보면 test set의 경우에도 sugar feature($x_2$)의 중요도가 높게 평가됨을 확인할 수 있다.

In [ ]:
result = permutation_importance(hgb, X_test, y_test,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


이제 최종적으로 test set에 대한 model performance를 확인하자.

In [ ]:
hgb.score(X_test, y_test)

0.8723076923076923